# AUA, DS 229 – MLOps
### Week 6 – Testing ML systems with `pytest` and `tox`

***

In [ ]:
# !pip install -r requirements.txt

### `==` vs `is`

- `==` is for value equality. It's used to know if two objects have the same value.
- `is` is for reference equality. It's used to know if two references refer (or point) to the same object, i.e if they're identical. Two objects are identical if they have the same memory address.

**Two objects having equal values are not necessarily identical.**  
For short, `==` determines if the values of two objects are equal, while `is` determines if they are the exact same object.

The `id()` function returns **a unique id for the specified object**. <mark>All objects in Python have their own unique ids. The id is assigned to the object when it is created and represents object's memory address.</mark>

In [ ]:
a = [1, 2, 3]
b = a

print(f"a == b: {a == b}")
print(f"a is b: {a is b}")
print("-"* 15, f"\nid(a): {id(a)}")
print(f"id(b): {id(b)}")
print(f"id(a) == id(b): {id(a) == id(b)}")

In [ ]:
c = a[:]  # c = [1, 2, 3]

print(f"a == c: {a == c}")
print(f"a is c: {a is c}")
print("-"* 15, f"\nid(a): {id(a)}")
print(f"id(c): {id(c)}")
print(f"id(a) == id(c): {id(a) == id(c)}")

#### Ambiguities

In [ ]:
a = 1000
b = 1000

print(f"a == b: {a == b}")
print(f"a is b: {a is b}")
print("-"* 15, f"\nid(a): {id(a)}")
print(f"id(b): {id(b)}")
print(f"id(a) == id(b): {id(a) == id(b)}")

In [ ]:
a = 256
b = 256

print(f"a == b: {a == b}")
print(f"a is b: {a is b}")
print("-"* 15, f"\nid(a): {id(a)}")
print(f"id(b): {id(b)}")
print(f"id(a) == id(b): {id(a) == id(b)}")

<mark>The reason for so is that Python caches integer objects in the range $[-5, 256]$ as singleton instances for performance reasons.</mark>

[PEP8](https://peps.python.org/pep-0008/#programming-recommendations) styleguide suggests that **comparisons to singletons like None should always be done with `is` or `is not`, never the equality operators**.

### [**pytest**](https://docs.pytest.org/en/7.2.x/)

The structure of today's toy project:

```
├── source/
│   ├── __init__.py
│   └── softmax.py
│
├── tests/
│   ├── __init__.py
│   └── test_softmax.py
│
├── pytest.ini
├── tox.ini
└── requirements.txt

```

Let's examine the **stable_softmax** function in `source/softmax.py`. It implements stable softmax function by taking inputs an array $x$ (either numpy or torch) and a constant value $const$ and applying softmax on $x-const$. For $x = [x_1, x_2, \dots, x_n]$, 

$$
\text{softmax}(x)_i = \frac{\exp{(x_i)}}{\sum_{k=1}^n \exp{(x_k)}},
$$

$$
\text{softmax}(x + c) = \text{softmax}(x),
$$

where $c$ is some constant. It is empirically proven that picking $const := -\max(x)$ helps avoiding numerical overflow. 

In our implementation of **stable_softmax** we keep `const` as a free argument for demonstrating certain `pytest` functionalities. 

<div class="alert alert-block alert-danger">
<b>Action</b>:
    Go through <b>source/softmax.py</b>.
</div> 

`pytest` is a framework for testing code written in python in a more flexible and easier manner compared to `unittest`.

When `pytest` is run in a terminal, it will search all directories below where it was called, find all of the Python files in these directories whose **names start with `test_` or end with `_test`**, import them, and run all of the functions and classes whose **names start with `test` or `Test`**.

However, to inject custom behavior, we can specify pytest settings either in `pytest.ini` or `pyproject.toml` files. These files by convention reside in the root directory of your repository.

In our codebase we have `pytest.ini` file of the following structure:
- **testpaths**: Sets list of directories that should be searched for tests when no specific directories, files or test ids are given in the command line when executing pytest from the rootdir directory. Useful when all project tests are in a known location to speed up test collection and to avoid picking up undesired tests by accident.
- **python_files**: One or more glob-style file patterns determining which python files are considered as test modules.
- **markers**: When the `--strict-markers` or `--strict command-line` arguments are used, only known markers defined in code by core pytest or some plugin are allowed. You can list additional markers in this setting to add them to the whitelist, in which case you probably want to add `--strict-markers` to avoid future regressions.

<div class="alert alert-block alert-danger">
<b>Action</b>:
    Uncomment and run <b>SIMPLE TEST</b> from <b>tests/test_softmax.py</b>.
</div> 

In [ ]:
!pytest --verbose

# !pytest tests/test_softmax.py --verbose
# !pytest tests/ --verbose
# !pytest tests/test_softmax.py::test_softmax_numpy_multiple_1d --verbose

<div class="alert alert-block alert-danger">
<b>Action</b>:
    Uncomment and run <b>MULTIPLE TESTS</b> from <b>tests/test_softmax.py</b>.
</div> 

In [ ]:
!pytest --verbose

<div class="alert alert-block alert-danger">
<b>Action</b>:
    Uncomment and run <b>MULTIPLE TESTS (PARAMETRIZATION)</b> from <b>tests/test_softmax.py</b>.
</div> 

The builtin **pytest.mark.parametrize** decorator enables parametrization of arguments for a test function. As a first argument it receives testing function argument names in a string separated by commas. Next, it expects a list of tuples where each tuple represents testing function argument values. Then the underlying testing function is run as many times as the number of tuples is.

In [ ]:
!pytest --verbose

<div class="alert alert-block alert-danger">
<b>Action</b>:
    Uncomment and run <b>MULTIPLE TESTS (COMBINATIONS)</b> from <b>tests/test_softmax.py</b>.
</div> 

When applied **pytest.mark.parametrize** decorators multiple times, the testing function is run for all combinations of arguments. 

In [ ]:
!pytest --verbose

<div class="alert alert-block alert-danger">
<b>Action</b>:
    Uncomment and run <b>BAD INPUTS</b> from <b>tests/test_softmax.py</b>.
</div> 

To write assertions about raised exceptions, you can use **pytest.raises()** as a **context manager**.

In [ ]:
!pytest --verbose

<div class="alert alert-block alert-danger">
<b>Action</b>:
    Uncomment and run <b>MULTIPLE TESTS (FIXTURES)</b> from <b>tests/test_softmax.py</b>.
</div> 

If you notice that you're creating multiple tests that **rely on the same set of data**, then it's likely that you'll need to use a fixture. Essentially, you can gather the shared data into a single function and decorate it with **@pytest.fixture** to signify that it's a fixture in pytest. This will allow you to use the same data as input for multiple test functions.

Pytest only caches one instance of a fixture at a time, which means that when using a parametrized fixture, pytest may invoke a fixture more than once in the given scope.

In [ ]:
!pytest --verbose

<div class="alert alert-block alert-danger">
<b>Action</b>:
    Run <b>pytest --cov source</b> to view the test coverage report.
</div> 

In [ ]:
!pytest --cov source

In [ ]:
!pytest --cov source --cov-report html  # This will generate a more detailed and user-friendly report.

In [ ]:
# Inside htmlcov directory you can find report files for each python script.
# Let's open the one for softmax.py:
!open htmlcov/d_411ffd8adc737496_softmax_py.html

<div class="alert alert-block alert-danger">
<b>Action</b>:
    Uncomment and run <b>PYTORCH INPUTS</b> from <b>tests/test_softmax.py</b> and check the coverage again.
</div> 

In [ ]:
!pytest --verbose

In [ ]:
!pytest --cov source

<center><img src="./images/dijkstra.jpeg"/></center>

<div class="alert alert-block alert-danger">
<b>Action</b>:
    Uncomment and run <b>MARKERS</b> from <b>tests/test_softmax.py</b>.
</div> 

In [ ]:
!pytest --verbose

To use custom markers correctly, we must specify the newly-created markers in the **pyproject.toml** file. We can use the **--strict-markers** flag to ensure that all markers are defined in this file. Then, we need to declare our markers in markers list, providing some description on them.

In [ ]:
!pytest --markers  # Lists all the defined markers.

In [ ]:
!pytest -m "small"  # Runs all tests that are marked with 'small'.

In [ ]:
!pytest -m "not small"  # Runs all tests that are NOT marked with 'small'.

### [**tox**](https://tox.wiki/en/latest/)

`tox` is used to **automate Python tests** in a Machine Learning pipelines or in general Python projects. It is an open-source project that provides a convenient way to run commands in isolated environments. What tox does in the background can be roughly split into four main steps:
1) Creates a virtual environment.
2) Installs dependencies in the virtual environment.
3) Runs commands.
4) Returns output (for each environment created).

<center><img src="./images/tox-diagram.png" width=900 height = 900/></center>

The steps 1 to 3 are run according to what the user provides in a config file. There are three approaches to configure tox. The first one is by adding a `tox.ini` file in the root of the project. The second one is by adding a `tool.tox` section in the `pyproject.toml` configuration file. The third one is by adding a `setup.cfg` file. We will move forward with `tox.ini` option.



<div class="alert alert-block alert-danger">
<b>Action</b>:
    Take a look at <b>tox.ini</b> file.
</div> 

Each executable block can be identified by square brackets. The first part is what we called global settings, which are contained in the first section called `[tox]`. Under that section there is an item `envlist`, which tells tox which environments to create when we run tox from the command line (step 1). The section also contains `skipsdist` item which is set to True when we are not testing a softawre package (don't have `setup.py`). The second section, `[testenv]`, tells tox what dependencies to install in our environments (step 2). This is specified in the `deps` variable. Then, with the environments we created and the dependencies installed, we tell tox to run he specified command for the step 3. In our example file we do the fllowing:
1) Create environments with Python 3.7.14 and Python 3.8.14.
2) Install *pytest, numpy, torch* and *scipy* in each of the environemnt. 
3) Tell tox to run `pytest --verbose`.


>Note that your device must support the versions of python listed in the first step. For example, since I mange my python environments with `pyenv`, I run the following commands to install Python 3.7.14 and Python 3.8.14 interpretators: 
>- `pyenv install -v 3.7.14`
>- `pyenv install -v 3.8.14`

To run a `tox.ini` file from the root of the working directory, we simply run `tox` from the terminal. This means that **tox will create the new environments**, **install the dependencies**, and **run the commands** provided in the configuration. Then it caches the environments to save time for later usage.

<div class="alert alert-block alert-danger">
<b>Action</b>:
    Run <b>tox</b>.
</div> 

In [ ]:
!tox

There are multiple variations of `tox` command, below are listed some of them:
- `tox -e python3.7.14`: run against specified environments (for example, Python 3.7.14).
- `tox -i https://pypi.my-alternative-index.org`: force to use an alternative URL address to download packages.
- `tox --parallel --recreate python3.7.14 python3.8.14`: create and run multiple virtual environments in parallel (for example, to run python3.7.14 and python3.8.14 in parallel).
- `tox --parallel-live --recreate python3.7.14 python3.8.14`: show the output of the parallel environments mentioned above.

<div class="alert alert-block alert-danger">
<b>Action</b>:
    Modify <b>tox.ini</b> file to install <b>requirements.txt</b> and run <b>tox</b> again.
</div> 

```
[tox]
envlist = python3.7.14, python3.8.14
skipsdist = true

[testenv]
deps = -rrequirements.txt
commands = pytest --verbose
```

In [ ]:
!tox

Within `tox.ini` one can define custom environments with their dependencies, run various commands (like in terminal), set environment variables, etc.

**<mark>IMPORTANT</mark>**  
Using an outdated tox virtual environment can pose a risk. Tox has a limitation in that it cannot detect modifications in the dependencies mentioned in the `setup.py` and/or `requirements.txt` files. It's important to keep this in mind and to recreate the environments by using the `tox -r` flag whenever such changes are made. This will ensure that the environments are refreshed and up-to-date.


In [ ]:
!tox -r

# References
- [Testing Machine Learning Systems: Code, Data and Models](https://madewithml.com/courses/mlops/testing/#behavioral-testing)
- [A simple tox.ini / default environments](https://tox.wiki/en/3.26.0/example/basic.html)